# Initializing chunking

In [101]:
file_name='input/md/hindi/Natural_Farming_Kharif_Booklet.md'
md_file=open(file_name,'r',encoding='utf-8')
md=md_file.read()
chunks=md.split('\n')

# chunks=[chunk for chunk in chunks if chunk!='']
# for chunk in chunks:
#     print(chunk,end='\n*****\n')

। |


In [102]:
chunks

['',
 '## प्राकृतिक तकनीक द्वारा बाजरा की',
 '',
 'प्रदेश में बाजरे की वर्ष 2022 के आंकड़े से लगभग 9.80 लाख हेक्टेयर है।',
 'प्रदेश में बाजरे की खेती से लगभग 50 लाख मीट्रिक टन उत्पादन प्राप्त हो रहा है। कम',
 'वर्षा',
 'मेंइसकी खेती सफल है 40-50 से०मी0 वार्षिक वर्षा वाले',
 'क्षेत्रों',
 'मेंइसकी खेती',
 'सफलता पूर्वक की जा सकती है। बाजरा व्यापक रूप से मानव भोजन एवं चारे के रूप',
 'में उगाया जाता',
 'हैं।',
 'प्रदेश',
 'में बाजरे की खेती बुन्देलखण्ड',
 'क्षेत्र, आगरा, बरेली, एवं',
 'कानपुर मण्डलों में होती है।',
 'प्रजातियों',
 'काचयनः अच्छी उपज प्राप्त करने लिए उननतिशील प्रजातियों का शुद्ध बीज',
 'ही बोना चाहिए। लेकिन कुछ किसमें ऐसी है जिन्हें लोग पशुओं के चारे के लिए उगाते हैं।',
 'उत्तर',
 'प्रदेश',
 'में बाजरे की',
 'प्रजातियों उपलब्ध',
 'है',
 'उनमें',
 'से',
 'प्रमुख है-',
 'पूसा',
 'कम्पोजिट-704, धन-शक्ति, पूसा कम्पोजिट-443',
 '|',
 'भूमि का चुनाव: बाजरा के लिए हल्की या दोमट बलुई मिट्टी उपयुक्त होती है। भूमि का जल निकास उत्तम होना आवश्यक है एवं जिस भूमि',
 'में पहली बार प्राकृति

In [103]:
def table_parser(chunks,index):
    inside_table = False
    current_table = ''
    for i,chunk in enumerate(chunks):
        if inside_table:
            # Check if the current chunk is the end of the tablere
            if chunk.startswith('|') :
                inside_table = True
                current_table=current_table+'\n'+chunk
            else:

                return current_table,(index+i)
        elif chunk.startswith('|'):
            inside_table = True
            current_table=chunk
    #     elif chunk=='':
    #         continue
#         else:
#             print('post',(index+i))
#             return combined_chunks,(index+i)
    return 0


def para_parser(chunks,index):
    inside_para=False
    current_para=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if chunk=='':
            i=i+1            
        elif inside_para:
            if chunk[0] not in temp:
                inside_para=True
                current_para=current_para+' '+chunk
            else:
                current_para=current_para.strip()
                # if len(current_para)==0:
                #     return(current_para,'flag')
                return (current_para,(index+i))            
        elif (chunk[0] not in temp) or chunk!='|':
            inside_para=True
            current_para=chunk
            
    return (current_para,(index+i))

def list_parser(chunks,index):
    inside_list=False
    current_list=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if inside_list:
            if chunk=='' or chunk.startswith('- ') or chunk.startswith('* '):
                return (current_list,(index+i))
            if chunk[0] not in temp:
                current_list=current_list+'\n'+chunk
        elif chunk.startswith('-') or chunk.startswith('* '):
            current_list=chunk
            inside_list=True

    return (current_list,(index+i))

# Extracting types for chunks and creating a JSON

In [109]:
# for i in range(len(chunks)):
i=0
combined_chunks=[]
temp=['#','## ','### ','#### ','##### ','###### ','-','*','|']
while i<len(chunks):
    if chunks[i]=='':
        combined_chunks.append({'type':'return','content':chunks[i]})
        i=i+1
    elif chunks[i].startswith('|') and len(chunks[i])!=1:
        chunk,i=table_parser(chunks[i:],i)
        combined_chunks.append({'type':'table','content':chunk})

    elif chunks[i].startswith('# ') or chunks[i].startswith('## ') or chunks[i].startswith('### ') or chunks[i].startswith('#### ') or chunks[i].startswith('##### '):
        combined_chunks.append({'type':'heading','content':chunks[i]})
        i=i+1

    elif chunks[i].startswith('- ') or chunks[i].startswith('* '):
        chunk,i=para_parser(chunks[i:],i)
        combined_chunks.append({'type':'list','content':chunk})

    elif (chunks[i][0] not in temp and (i+1)!=len(chunks)) or chunks[i]=='|':
        chunk,i=para_parser(chunks[i:],i)
            
            # chunk=chunks[i].strip()
        if len(chunk)==0:
                
                continue
        combined_chunks.append({'type':'para','content':chunk})
            # i=i+1
    else:
        i=i+1

In [110]:
combined_chunks = [chunk for chunk in combined_chunks if chunk['type'] != 'return']
# combined_chunks

In [111]:
combined_chunks

[{'type': 'heading', 'content': '## प्राकृतिक तकनीक द्वारा बाजरा की'},
 {'type': 'para',
  'content': 'प्रदेश में बाजरे की वर्ष 2022 के आंकड़े से लगभग 9.80 लाख हेक्टेयर है। प्रदेश में बाजरे की खेती से लगभग 50 लाख मीट्रिक टन उत्पादन प्राप्त हो रहा है। कम वर्षा मेंइसकी खेती सफल है 40-50 से०मी0 वार्षिक वर्षा वाले क्षेत्रों मेंइसकी खेती सफलता पूर्वक की जा सकती है। बाजरा व्यापक रूप से मानव भोजन एवं चारे के रूप में उगाया जाता हैं। प्रदेश में बाजरे की खेती बुन्देलखण्ड क्षेत्र, आगरा, बरेली, एवं कानपुर मण्डलों में होती है। प्रजातियों काचयनः अच्छी उपज प्राप्त करने लिए उननतिशील प्रजातियों का शुद्ध बीज ही बोना चाहिए। लेकिन कुछ किसमें ऐसी है जिन्हें लोग पशुओं के चारे के लिए उगाते हैं। उत्तर प्रदेश में बाजरे की प्रजातियों उपलब्ध है उनमें से प्रमुख है- पूसा कम्पोजिट-704, धन-शक्ति, पूसा कम्पोजिट-443'},
 {'type': 'para',
  'content': 'भूमि का चुनाव: बाजरा के लिए हल्की या दोमट बलुई मिट्टी उपयुक्त होती है। भूमि का जल निकास उत्तम होना आवश्यक है एवं जिस भूमि में पहली बार प्राकृतिक खेती करनी है उस मिट्टी की

In [ ]:
inside_pair=False
paired_chunks=[]
current_pair={'type':'heading-para pair','content':{}}
for chunk in combined_chunks:
    if chunk['type']=='heading':
        inside_pair=True
        current_pair['content']['heading']=chunk['content']
    elif chunk['type']=='para':
        if inside_pair:
            current_pair['content']['para']=chunk['content']
            paired_chunks.append(current_pair)
            inside_pair=False
            current_pair={'type':'heading-para pair','content':{}}
        else:
            paired_chunks.append(chunk)
    else:
        paired_chunks.append(chunk)

In [ ]:
# def count_level(content):
#     level
#     if content.startswith('# '):
#         level=1
#     elif content.startswith('## '):
#         level=2
#     elif content.startswith('### '):
#         level=3
#     elif content.startswith('#### '):
#         level=4
#     elif content.startswith('##### '):
#         level=5
#     elif content.startswith('###### '):
#         level=6


#     return level

# Creating heading heirarchy

In [112]:
def assign_ids_and_parents(chunks):
    id_counter = {'1': 0, '2': 0, '3': 0,'4':0,'5':0}
    parent_stack = []

    for chunk in chunks:
        if chunk['type'] == 'heading':
            level = chunk['content'].count('#')
#             level=count_level(chunk['content'])
#             print('hehe',level)
            id_counter[str(level)] += 1
            chunk['id'] = f'{level}{id_counter[str(level)]}'

            if parent_stack==[] or level==1:
                parent_stack = [chunk['id']]
            else:
                buff=int(str(parent_stack[0])[0])-1
                while len(parent_stack)>=(level-buff):
                    parent_stack.pop()
                parent_stack.append(chunk['id'])
            chunk['parents'] = parent_stack[:-1]
#             print(parent_stack)

        else:
            chunk['parents'] = parent_stack.copy() if parent_stack else []

    return chunks
final_chunks=assign_ids_and_parents(combined_chunks)
final_chunks

[{'type': 'heading',
  'content': '## प्राकृतिक तकनीक द्वारा बाजरा की',
  'id': '21',
  'parents': []},
 {'type': 'para',
  'content': 'प्रदेश में बाजरे की वर्ष 2022 के आंकड़े से लगभग 9.80 लाख हेक्टेयर है। प्रदेश में बाजरे की खेती से लगभग 50 लाख मीट्रिक टन उत्पादन प्राप्त हो रहा है। कम वर्षा मेंइसकी खेती सफल है 40-50 से०मी0 वार्षिक वर्षा वाले क्षेत्रों मेंइसकी खेती सफलता पूर्वक की जा सकती है। बाजरा व्यापक रूप से मानव भोजन एवं चारे के रूप में उगाया जाता हैं। प्रदेश में बाजरे की खेती बुन्देलखण्ड क्षेत्र, आगरा, बरेली, एवं कानपुर मण्डलों में होती है। प्रजातियों काचयनः अच्छी उपज प्राप्त करने लिए उननतिशील प्रजातियों का शुद्ध बीज ही बोना चाहिए। लेकिन कुछ किसमें ऐसी है जिन्हें लोग पशुओं के चारे के लिए उगाते हैं। उत्तर प्रदेश में बाजरे की प्रजातियों उपलब्ध है उनमें से प्रमुख है- पूसा कम्पोजिट-704, धन-शक्ति, पूसा कम्पोजिट-443',
  'parents': ['21']},
 {'type': 'para',
  'content': 'भूमि का चुनाव: बाजरा के लिए हल्की या दोमट बलुई मिट्टी उपयुक्त होती है। भूमि का जल निकास उत्तम होना आवश्यक है एवं जिस

In [8]:
len(final_chunks)

38

# Creating CSV

In [113]:
headings={}
for chunk in final_chunks:
    if chunk['type']=='heading':
        headings[chunk['id']]=chunk['content']

In [138]:
def get_subheading(chunks,index):
  subheadings=''
  for heading in chunks[index]['parents']:
    subheadings+=(' '.join(headings[heading].split()[1:]))+','
  return subheadings[:-1]

import csv
#CSV columns: chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
def export_csv(final_chunks):

  with open('output.csv','w') as file:
    csv_writer = csv.writer(file)
    data=['chunkId','content','StartPage','EndPage','heading','ContentWordCount','pdfName','contentType','image','summary']
    csv_writer.writerow(data)

  for i,chunk in enumerate(final_chunks):
    chunkId=i+1
    content=chunk['content']
    heading=get_subheading(final_chunks,i)
    contentType=chunk['type']
    ContentWordCount=len(content.split()) if contentType!='table' else content.split('|')
    pdfName=file_name.split('.')[0]+'.pdf'
    data=[chunkId,content,'','',heading,ContentWordCount,pdfName,contentType,'','']
    with open('output.csv','a',encoding='utf-8') as file:
      csv_writer = csv.writer(file)
      csv_writer.writerow(data)




In [139]:
original_string = "This is a simple string."

# Split the string into words, add a space after each word, and join them
modified_string = ' '.join(original_string.split()[1:])

# Display the modified string
print(modified_string)

is a simple string.


In [140]:
export_csv(final_chunks)

# Translating

In [84]:
import requests
import json
import pandas as pd
import time

def translate_text_bhashini(input_text):
    url = "https://dhruva-api.bhashini.gov.in/services/inference/pipeline"

    payload = json.dumps({
        "pipelineTasks": [
            {
                "taskType": "translation",
                "config": {
                    "language": {
                        "sourceLanguage": "hi",
                        "targetLanguage": "en"
                    },
                    "serviceId": "ai4bharat/indictrans-v2-all-gpu--t4"
                }
            }
        ],
        "inputData": {
            "input": [
                {
                    "source": input_text
                }
            ]
        }
    })
    headers = {
        'Accept': '*/*',
        'User-Agent': 'Thunder Client (https://www.thunderclient.com)',
        'Authorization': 'sLAFJehUCZQ72NIM4nDZNCya7TQVzittLgJEU0vIf-69rp0gFUcGu5sjwAaOSUfa',
        'Content-Type': 'application/json'
    }

    retries = 0
    max_retries = 5
    start_time = time.time()
    while retries < max_retries:
        try:
            response = requests.post(url, headers=headers, data=payload)
            # Check if the request was successful
            if response.status_code == 200:
                translated_output = json.loads(response.text)['pipelineResponse'][0]['output'][0]['target']
                end_time = time.time()
                return translated_output, retries, end_time - start_time
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                retries += 1
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")
            retries += 1

    end_time = time.time()
    return "Translation failed after maximum retries.", max_retries, end_time - start_time

In [86]:
a

('The area under millets in the state is about 9.8 lakh hectares as of 2022. About 50 lakh metric tonnes of produce is being obtained from millet cultivation in the state. It is successful in low rainfall. It can be successfully cultivated in areas with 40-50 cm annual rainfall. Millets are widely grown as human food and fodder. In the state, millet is cultivated in the Bundelkhand region, Agra, Bareilly, and Kanpur divisions. Cultivation of species: Only pure seeds of non-degradable species should be sown to get good yield. But there are some varieties that people grow for animal fodder. Major millet varieties available in Uttar Pradesh are Pusa Composite-704, Dhan-Shakti, Pusa Composite-443.',
 0,
 0.6947267055511475)

In [124]:
import os
input_folder='output\\hindi'
all_files=[]
for folder, subfolders, files in os.walk(input_folder):
    full_file_paths = [os.path.join(folder, file) for file in files]
    all_files.extend(full_file_paths)
all_files

['output\\hindi\\Natural_Farming_Kharif_Booklet.csv',
 'output\\hindi\\Pashupalan Margdarshika.csv',
 'output\\hindi\\RabiBookallpagesfullbook_2019.csv']

In [143]:
import pandas as pd
df=pd.read_csv(all_files[0])
newdf=pd.read_csv('output.csv')

In [142]:
newdf

,chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
0,1,## प्राकृतिक तकनीक द्वारा बाजरा की,NaN,NaN,NaN,6,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,heading,NaN,NaN
1,2,प्रदेश में बाजरे की वर्ष 2022 के आंकड़े से लगभ...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,125,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
2,3,भूमि का चुनाव: बाजरा के लिए हल्की या दोमट बलुई...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,39,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
3,4,बुआई का समय एवं तापमान: वर्षा प्रारम्भ होते ही...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,314,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
4,5,जीवामृत: बाजरा की फसल में पोषक तत्वों की पूर्त...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,286,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
60,61,## प्राकृतिक तकनीक द्वारा तिल की,NaN,NaN,प्राकृतिक तकनीक द्वारा अरहर की,6,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,heading,NaN,NaN
61,62,"प्रदेश मेंतिल की मुख्यतः बुन्देलखण्ड, मिर्जापु...",NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",369,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
62,63,सिंचाईं: तिल की फसलें जलभराव के प्रति संवदेनशी...,NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",298,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
63,64,कीट: 4. पत्ती व फल की सूंडी: इस की सूँडियाँ को...,NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",56,input/md/hindi/Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN


In [147]:
df = df.rename(columns={'content': 'originalLanguageChunk','heading':'originalLanguageHeading'})

In [151]:
print('translating')
df['content'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageChunk'])[0] if row['contentType'] != 'table' else row['originalLanguageChunk'], axis=1)
df['heading'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageHeading'])[0] if row['contentType'] != 'table' else row['originalLanguageHeading'], axis=1)
print('\rtranslating done')
df=df[]

translating
translating done


In [154]:
df=df[['chunkId','content','heading','originalLanguageChunk','StartPage','EndPage','originalLanguageHeading','ContentWordCount','pdfName','contentType','image','summary']]

In [155]:
df

,chunkId,content,heading,originalLanguageChunk,StartPage,EndPage,originalLanguageHeading,ContentWordCount,pdfName,contentType,image,summary
0,1,# # Millet by natural technique,nan,## प्राकृतिक तकनीक द्वारा बाजरा की,NaN,NaN,NaN,6,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,heading,NaN,NaN
1,2,The area under millets in the state is about 9...,Millet by natural technique.,प्रदेश में बाजरे की वर्ष 2022 के आंकड़े से लगभ...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,125,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
2,3,Land selection: Light or loamy sandy soil is s...,Millet by natural technique.,भूमि का चुनाव: बाजरा के लिए हल्की या दोमट बलुई...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,39,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
3,4,Sowing Time and Temperature: As soon as the ra...,Millet by natural technique.,बुआई का समय एवं तापमान: वर्षा प्रारम्भ होते ही...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,314,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
4,5,Jeevamrut: Jeevamrut is sprayed to replenish t...,Millet by natural technique.,जीवामृत: बाजरा की फसल में पोषक तत्वों की पूर्त...,NaN,NaN,प्राकृतिक तकनीक द्वारा बाजरा की,286,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
60,61,# # of sesame by natural technique,of arhar by natural technique.,## प्राकृतिक तकनीक द्वारा तिल की,NaN,NaN,प्राकृतिक तकनीक द्वारा अरहर की,6,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,heading,NaN,NaN
61,62,"In the state, til is mainly cultivated as pure...","Arhar by natural technique, Til by natural tec...","प्रदेश मेंतिल की मुख्यतः बुन्देलखण्ड, मिर्जापु...",NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",369,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
62,63,Irrigation: Sesame crops are susceptible to wa...,"Arhar by natural technique, Til by natural tec...",सिंचाईं: तिल की फसलें जलभराव के प्रति संवदेनशी...,NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",298,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN
63,64,Insects: 4. Leaf-and-fruit sundaes: Its sundae...,"Arhar by natural technique, Til by natural tec...",कीट: 4. पत्ती व फल की सूंडी: इस की सूँडियाँ को...,NaN,NaN,"प्राकृतिक तकनीक द्वारा अरहर की,प्राकृतिक तकनीक...",56,input/md/hindi\Natural_Farming_Kharif_Booklet.pdf,para,NaN,NaN


# Splitting bigger chunks

In [6]:
import re
import pandas as pd

In [7]:
df=pd.read_csv('output/english\\FARMER PRODUCER ORGANISATIONS.csv')

In [67]:
new_rows = []
count=0
for index, row in df.iterrows():
    
    print('**********')
    word_count = row['ContentWordCount']
    content = row['content']
    
    # Check if ContentWordCount is greater than 200
    if int(word_count) > 200 and row['contentType']=='para':
        # Split the content into chunks of <= 200 words with complete sentences
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
        
        
        chunks = []
        current_chunk = []
        for sentence in sentences:
            # print(sentence)
            
            temp=' '.join(current_chunk + [sentence])
            if len(temp.split()) <= 200:
                current_chunk.append(sentence)
                # print(current_chunk)
                
            else:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
        print(current_chunk)
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        # Create new rows for each chunk
        print('reched',chunks)
        for chunk in chunks:
            count+=1
            new_row = {
                'chunkId': index,  # You may need to assign a new chunkId
                'content': chunk,
                'heading': row['heading'],
                'originalLanguageChunk': row['originalLanguageChunk'],
                'StartPage': row['StartPage'],
                'EndPage': row['EndPage'],
                'originalLanguageHeading': row['originalLanguageHeading'],
                'ContentWordCount': len(chunk.split()),
                'pdfName': row['pdfName'],
                'contentType': row['contentType'],
                'image': row['image'],
                'summary': row['summary']
            }
            new_rows.append(new_row)
    else:
        # If ContentWordCount is <= 200, keep the row as it is
        count+=1
        new_rows.append({
            'chunkId': count,
            'content': content,
            'heading': row['heading'],
            'originalLanguageChunk': row['originalLanguageChunk'],
            'StartPage': row['StartPage'],
            'EndPage': row['EndPage'],            
            'originalLanguageHeading': row['originalLanguageHeading'],
            'ContentWordCount': word_count,
            'pdfName': row['pdfName'],
            'contentType': row['contentType'],
            'image': row['image'],
            'summary': row['summary']
        })

# Create a new DataFrame with the updated rows
new_df = pd.DataFrame(new_rows)


In [68]:
new_d=split_chunks(df)

**********
**********
**********
**********
**********
**********
**********
**********
['Also, the initiatives taken and efforts put in by Dr. P Radhakrishnan, Chief General Manager, Dr. A R Khan and Shri Sanjay Kumar Dora, Deputy General Managers in coordinating with BIRD and other organizations/ departments in bringing out this manual, are greatly acknowledged.', 'Dr. B G Mukhopadhyay Chief General Manager Farm Sector Development Department NABARD Head Office, Mumbai     The Objective of this Manual is to act as a guide to Producer Organisation Promoting Institutions (POPIs) in Promoting and Capacity Building of Producer Organisations (POs) and help POs towards better Management and Business Development.']
reched ["and Rural Development National Bank for Agriculture Mumbai  2015 Title                               :                                  Farmer Producer Organisations - Frequently Asked Questions (FAQs)   Written and Published by                               :            

In [69]:
new_d

,chunkId,content,heading,StartPage,EndPage,ContentWordCount,pdfName,contentType,image,summary
0,1,## Nabard Farmer Producer Organisations Freque...,NaN,NaN,NaN,9,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,NaN
1,2,# National Bank For Agriculture Mumbai 2015,NaN,NaN,NaN,7,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,NaN
2,3,"""The issue of profitability of small holding b...",National Bank For Agriculture Mumbai 2015,NaN,NaN,75,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,NaN
3,4,# And Rural Development,NaN,NaN,NaN,4,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,NaN
4,5,## Nabard'S Mission,And Rural Development,NaN,NaN,3,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
713,714,##,(On Stamp Paper Of Appropriate Value As Per Th...,NaN,NaN,1,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,NaN
714,568,For NABARD For RSA In witness thereof the p...,(On Stamp Paper Of Appropriate Value As Per Th...,NaN,NaN,33,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,NaN
715,568,Attachment 7 - Abbreviations Abbreviation **...,(On Stamp Paper Of Appropriate Value As Per Th...,NaN,NaN,179,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,NaN
716,717,| Abbreviation | ...,(On Stamp Paper Of Appropriate Value As Per Th...,NaN,NaN,253,input/md/english\FARMER PRODUCER ORGANISATIONS...,table,NaN,NaN


In [33]:
sentence="and Rural Development National Bank for Agriculture Mumbai  2015 Title                               :                                  Farmer Producer Organisations - Frequently Asked Questions (FAQs)   Written and Published by                               :                                  Farm Sector Policy Department & Farm Sector                                   Development Department, NABARD Head Office, Mumbai  Date of Publishing                               :                                  March 2015 Design & Printing                               :                                  M/s Image Impression Contact                               :                                  Plot No C-24, 'G' Block, Bandra Kurla Complex,                                  Bandra East, Mumbai - 400051."

In [34]:
a=' '.join(current_chunk + [sentence])

In [36]:
len(a)

805

# Summary

In [80]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key=os.environ.get("API_KEY")

def get_gpt_response(system_prompt,user_prompt):
  response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-1106", temperature = 0,
                messages=[{"role": "system", "content": system_prompt},
                          {"role": "user", "content": user_prompt}
                ])
  gpt_response =  response["choices"][0]["message"]["content"]
  return(gpt_response)

In [94]:

system_prompt = """You are an AI bot that summarises user provided content in 5 sentences.
The content that you are shared is part of the Samagra Processes and Policy Corpus. The data was extracted from the PDF and has two sections - Heading and Content.
These will be provided to you to summarize in the folloowing format

Heading: <Heading>
Content: <Content>

You dont say anything else apart from the 5 sentence summary. Do not reproduce either the Heading or Content.
"""
file='E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv'
df = pd.read_csv(file)

df['summary'] = ''
for i in range(df.shape[0]):
    print(i)
    if df.loc[i,'contentType']!='heading':
        content = "Heading:" + str(df.loc[i,'heading']) + "\n" + "Content: " + str(df.loc[i,'content'])
    
        df.loc[i,'summary'] =  get_gpt_response(system_prompt,content)
df.to_csv(file)

0
1
2


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [91]:
# summary('E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv')
df.loc[0,'contentType']

'heading'

In [87]:
df

,chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
0,1,## Nabard Farmer Producer Organisations Freque...,NaN,NaN,NaN,9,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,
1,2,# National Bank For Agriculture Mumbai 2015,NaN,NaN,NaN,7,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,
2,3,"""The issue of profitability of small holding b...",NaN,NaN,National Bank For Agriculture Mumbai 2015,75,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,
3,4,# And Rural Development,NaN,NaN,NaN,4,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,
4,5,## Nabard'S Mission,NaN,NaN,And Rural Development,3,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,
...,...,...,...,...,...,...,...,...,...,...
566,567,i. NABARD will monitor the progress through s...,NaN,NaN,(On Stamp Paper Of Appropriate Value As Per Th...,407,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,
567,568,##,NaN,NaN,(On Stamp Paper Of Appropriate Value As Per Th...,1,input/md/english\FARMER PRODUCER ORGANISATIONS...,heading,NaN,
568,569,For NABARD For RSA In witness thereof the p...,NaN,NaN,(On Stamp Paper Of Appropriate Value As Per Th...,212,input/md/english\FARMER PRODUCER ORGANISATIONS...,para,NaN,
569,570,| Abbreviation | ...,NaN,NaN,(On Stamp Paper Of Appropriate Value As Per Th...,253,input/md/english\FARMER PRODUCER ORGANISATIONS...,table,NaN,


# DUMP

In [ ]:
combined_chunks = []
inside_table = False
current_table = ''

for chunk in chunks:
    if inside_table:
        # Check if the current chunk is the end of the table
        if chunk.startswith('|'):
            inside_table = True
            current_table=current_table+'\n'+chunk
        else:
            combined_chunks.append(current_table)
            combined_chunks.append(chunk)
            inside_table=False
    elif chunk.startswith('|'):
        inside_table = True
        current_table=chunk
#     elif chunk=='':
#         continue
    else:
        combined_chunks.append(chunk)

In [ ]:
inside_pair=False
current_pair=''
for chunk in combined_chunks:
    if chunk['type']=='heading':
        iside_pair=True
    elif chunk['type']=='para':
        if inside_pair

In [ ]:
for i in combined_chunks:
    print(i,end='\n************\n')

## GPT Stuff (No use)

In [ ]:
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup

In [ ]:
file='Value_chain_financing.md'
file=open(file,'r',encoding='utf-8')
md_text=file.read()

In [ ]:
from markdown_it import MarkdownIt

def md_to_html(md_file_path, html_file_path):
    # Read Markdown content from file
    with open(md_file_path, 'r', encoding='utf-8') as md_file:
        md_text = md_file.read()
    md = MarkdownIt()
    html_content = md.render(md_text)
    with open(html_file_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)

md_path = "Value_chain_financing.md"
html_path = "output.html"
md_to_html(md_path, html_path)


In [ ]:
file=open('output.html', 'r', encoding='utf-8')
html_content = file.read()
page_data = {}
current_main_heading = ''
current_subheading = ''
crop_soup = BeautifulSoup(html_content, 'html.parser')
# print(crop_soup)
tags = crop_soup.find_all(['h4','h1','h2','h3', 'p', 'li','table'])

for tag in tags:
    if tag.name == 'h4':
        current_main_heading = tag.text.strip()
        current_subheading = ''  # Reset subheading when a new main heading is found
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
    elif tag.name == 'p' and tag.find('strong'):
        current_subheading = tag.text.strip()
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
        if current_subheading not in page_data[current_main_heading]:
            page_data[current_main_heading][current_subheading] = []
    elif current_main_heading and current_subheading:
        # Only add content if both main heading and subheading are available
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


    else:
        if current_main_heading and not current_subheading:
            current_subheading='General'
            page_data[current_main_heading][current_subheading]=[]
        if not current_main_heading and not current_subheading:
            current_main_heading='General'
            current_subheading='General'
            page_data[current_main_heading]={}
            page_data[current_main_heading][current_subheading]=[]
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


In [ ]:
page_data

In [ ]:
md = MarkdownIt()
tokens = md.parse(md_text)

headers = []
current_header = None

for token in tokens:
    print(token,end='\n\n\n')